# Azure AI Search з інтеграцією NVIDIA NIM та LlamaIndex

У цьому ноутбуці ми продемонструємо, як використовувати AI-моделі NVIDIA та LlamaIndex для створення потужного конвеєра Retrieval-Augmented Generation (RAG). Ми будемо використовувати великі мовні моделі (LLM) та ембеддинги NVIDIA, інтегрувати їх з Azure AI Search як сховище векторів і виконувати RAG для покращення якості та ефективності пошуку.

## Переваги
- **Масштабованість**: Використовуйте великі мовні моделі NVIDIA та Azure AI Search для масштабованого та ефективного пошуку.
- **Економічність**: Оптимізуйте пошук і отримання даних за допомогою ефективного зберігання векторів і гібридних методів пошуку.
- **Висока продуктивність**: Поєднуйте потужні LLM з векторизованим пошуком для швидших і точніших відповідей.
- **Якість**: Забезпечуйте високу якість пошуку, підкріплюючи відповіді LLM релевантними документами.

## Попередні вимоги
- 🐍 Python 3.9 або новіший
- 🔗 [Сервіс Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Ключ API NVIDIA для доступу до LLM та ембеддингів NVIDIA через мікросервіси NVIDIA NIM

## Охоплені функції
- ✅ Інтеграція LLM NVIDIA (ми будемо використовувати [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Ембеддинги NVIDIA (ми будемо використовувати [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Розширені режими отримання даних Azure AI Search
- ✅ Індексація документів за допомогою LlamaIndex
- ✅ RAG з використанням Azure AI Search та LlamaIndex разом із LLM NVIDIA

Почнемо!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Встановлення та вимоги
Створіть середовище Python, використовуючи версію Python >3.10.

## Початок роботи!


Щоб почати, вам потрібен `NVIDIA_API_KEY` для використання моделей NVIDIA AI Foundation:
1) Створіть безкоштовний обліковий запис на [NVIDIA](https://build.nvidia.com/explore/discover).
2) Натисніть на модель, яку ви обрали.
3) У розділі Input виберіть вкладку Python, натисніть **Get API Key**, а потім **Generate Key**.
4) Скопіюйте та збережіть створений ключ як NVIDIA_API_KEY. Після цього ви матимете доступ до кінцевих точок.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Приклад RAG із використанням LLM та Embedding
### 1) Ініціалізація LLM
`llama-index-llms-nvidia`, також відомий як конектор LLM від NVIDIA, дозволяє підключатися до сумісних моделей і генерувати результати за допомогою моделей, доступних у каталозі API від NVIDIA. Список моделей для завершення чатів можна знайти тут: https://build.nvidia.com/search?term=Text-to-Text

У цьому прикладі ми використаємо **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Ініціалізація Вбудовування  
`llama-index-embeddings-nvidia`, також відомий як конектор Вбудовувань NVIDIA, дозволяє підключатися до сумісних моделей і генерувати з них, доступних у каталозі API NVIDIA. Ми обрали `nvidia/nv-embedqa-e5-v5` як модель для вбудовування. Перегляньте список моделей для текстового вбудовування тут: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Створіть механізм запитів для отримання відповідей на основі ваших даних

Ось приклад запиту, який використовує чистий пошук за векторами в Azure AI Search і закріплює відповідь за нашим LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Ось запит із використанням гібридного пошуку в Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Аналіз пошуку вектора
Відповідь LLM точно передає ключові економічні наслідки, згадані в джерельному тексті щодо російського фондового ринку. Зокрема, зазначено, що російський фондовий ринок зазнав значного падіння, втративши 40% своєї вартості, а торгівлю було призупинено через поточну ситуацію. Ця відповідь добре узгоджується з інформацією, наведеною в джерелі, що свідчить про те, що LLM правильно визначив і підсумував відповідні деталі щодо впливу на фондовий ринок внаслідок дій Росії та накладених санкцій.

#### Коментарі до джерельних вузлів
Джерельні вузли надають детальний опис економічних наслідків, з якими зіткнулася Росія через міжнародні санкції. У тексті підкреслюється, що російський фондовий ринок втратив 40% своєї вартості, а торгівлю було призупинено. Крім того, згадуються інші економічні наслідки, такі як девальвація рубля та ширша ізоляція російської економіки. Відповідь LLM ефективно виділила ключові моменти з цих вузлів, зосереджуючись на впливі на фондовий ринок, як було запитано в запиті.


Тепер розглянемо запит, у якому Гібридний пошук не надає обґрунтованої відповіді:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Гібридний пошук: Аналіз відповіді LLM
Відповідь LLM у прикладі гібридного пошуку вказує на те, що наданий контекст не містить точної дати вторгнення Росії в Україну. Ця відповідь свідчить про те, що LLM використовує доступну інформацію з джерельних документів, але визнає відсутність конкретних деталей у тексті.

Відповідь є точною у визначенні того, що контекст згадує події, пов’язані з агресією Росії, але не уточнює конкретну дату вторгнення. Це демонструє здатність LLM розуміти надану інформацію, водночас визнаючи прогалини в змісті. LLM ефективно спонукає користувача звернутися до зовнішніх джерел або історичних записів для отримання точної дати, проявляючи обережність у випадках, коли інформація є неповною.

### Аналіз вузлів джерела
Вузли джерела у прикладі гібридного пошуку містять уривки з промови, яка обговорює реакцію США на дії Росії в Україні. Ці вузли акцентують увагу на ширшому геополітичному впливі та заходах, вжитих США та їх союзниками у відповідь на вторгнення, але не згадують конкретну дату вторгнення. Це узгоджується з відповіддю LLM, яка правильно визначає, що контекст не містить точної інформації про дату.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Гібридний підхід з повторним ранжуванням: Аналіз відповіді LLM
У прикладі з гібридним підходом і повторним ранжуванням відповідь LLM надає додатковий контекст, зазначаючи, що подія сталася за шість днів до виступу. Це свідчить про те, що LLM здатна зробити висновок про дату вторгнення, спираючись на час виступу, хоча для точності все ще необхідно знати точну дату виступу.

Ця відповідь демонструє покращену здатність використовувати контекстуальні підказки для надання більш інформативної відповіді. Вона підкреслює перевагу повторного ранжування, коли LLM може отримати доступ до більш релевантної інформації та пріоритизувати її, щоб надати більш точне наближення до потрібної деталі (тобто дати вторгнення).

### Аналіз вузлів джерела
Вузли джерела в цьому прикладі включають посилання на час вторгнення Росії, зокрема зазначаючи, що воно сталося за шість днів до виступу. Хоча точна дата все ще не вказана явно, вузли надають часовий контекст, який дозволяє LLM дати більш нюансовану відповідь. Включення цієї деталі демонструє, як повторне ранжування може покращити здатність LLM витягувати та робити висновки з наданого контексту, що призводить до більш точної та інформативної відповіді.


**Примітка:**  
У цьому блокноті ми використовували мікросервіси NVIDIA NIM з каталогу API NVIDIA.  
Вищезгадані API, `NVIDIA (llms)`, `NVIDIAEmbedding`, і [Azure AI Search Semantic Hybrid Retrieval (вбудоване повторне ранжування)](https://learn.microsoft.com/azure/search/semantic-search-overview). Зверніть увагу, що ці API також можуть підтримувати самостійно розміщені мікросервіси.  

**Приклад:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
